In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# aggregate energy results for each control strategy
powertrain_tech = "ICEV"
petroleum_equivalent_factor = 28996 # old value is 23160
fueltype = ["ICEV_fuel", "BEV_energy", "PHEV_electric_power", "PHEV_fuel", "HFCV_energy", "hev_fuel"]
file_dir = "results/"
control_method = ["No_control", "Signal_optimization_only", "Eco-Driving_only", "Proposed_control"]
demand_perct = [10, 50, 100]
cav_perct = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
energy_df = pd.DataFrame(columns=['control_method', 'demand_percent', 'cav_percent', 'average_energy_consumption'])

for control_strategy in control_method:
    for demand in demand_perct:
        for cav in cav_perct:
            files = "results/" + control_strategy + "_demand" + str(demand) + "_CAV" + str(cav) + '*.csv'
            avg_energy_per_km = 0
            num_repli = 0

            for repli in files:
                df = pd.read_csv(repli, delimiter='\t')
                num_repli += 1
                veh_ids = df.vehicle_id.unique()
                num_vehicles = len(veh_ids)
                energy_per_km = 0
                for idx in veh_ids:
                    df_by_vehicle = df[df['vehicle_id']==idx]
                    cumulative_distance = df_by_vehicle['driving_distance'].sum()
                    
                    if powertrain_tech == "ICEV":
                        total_energy = df_by_vehicle[fueltype[0]].sum()
                    elif powertrain_tech == "BEV":
                        total_energy = df_by_vehicle[fueltype[1]].sum()
                    elif powertrain_tech == "PHEV":
                        total_energy_1 = df_by_vehicle[fueltype[2]].sum()
                        total_energy_2 = df_by_vehicle[fueltype[3]].sum()
                        total_energy = total_energy_1 + total_energy_2 / petroleum_equivalent_factor
                    elif powertrain_tech == "HFCV":
                        total_energy = df_by_vehicle[fueltype[4]].sum()
                    elif powertrain_tech == "HEV":
                        total_energy = df_by_vehicle[fueltype[5]].sum()

                    energy_per_km += total_energy / cumulative_distance * 1000
                
                avg_energy_per_km += energy_per_km / num_vehicles
            
            repliAvg_energy_per_km = avg_energy_per_km / num_repli
            df1 = pd.DataFrame({'control_method': control_strategy, 'demand_percent' : demand, 'cav_percent': cav, 'average_energy_consumption': repliAvg_energy_per_km})
            energy_df = energy_df.append(df1, ignore_index = True)

energy_df.to_csv('agg_results/average_energy_results.csv', sep='\t')



In [ ]:
# calculate energy savings for each control strategy
energy_saving = pd.DataFrame(columns=['demand', 'cav', 'signal_optimization', 'eco-drive', 'cooperative_control'])

for demand in demand_perct:
    for cav in cav_perct:
        energy_no_control = energy_df.loc[(energy_df['control_method']=='No_control' & energy_df['demand_percent']==demand
                                          & energy_df['cav_percent']==cav), 'average_energy_consumption'].iloc[0]
        energy_sig_opt = energy_df.loc[(energy_df['control_method']=='Signal_optimization_only' & energy_df['demand_percent']==demand
                                          & energy_df['cav_percent']==cav), 'average_energy_consumption'].iloc[0]
        energy_eco_drive = energy_df.loc[(energy_df['control_method']=='Driving_only' & energy_df['demand_percent']==demand
                                          & energy_df['cav_percent']==cav), 'average_energy_consumption'].iloc[0]
        energy_cooperative_control = energy_df.loc[(energy_df['control_method']=='Proposed_control' & energy_df['demand_percent']==demand
                                          & energy_df['cav_percent']==cav), 'average_energy_consumption'].iloc[0]
        energy_saving_sig_opt = (energy_no_control - energy_sig_opt) / energy_no_control * 100
        energy_saving_eco_drive = (energy_no_control - energy_eco_drive) / energy_no_control * 100
        energy_saving_cooperative_control = (energy_no_control - energy_cooperative_control) / energy_no_control * 100
        df2 = pd.DataFrame({'demand': demand, 'cav': cav, 'signal_optimization': energy_saving_sig_opt, 'eco-drive': energy_saving_eco_drive, 'cooperative_control': energy_saving_cooperative_control})
        energy_saving.append(df2, ignore_index = True)
    
    energy_saving.to_csv('results/energy_saving.csv')


In [ ]:
# results visualization
demand_perct = [10, 50, 100]

# energy_saving vs. cav_percent
cav_perct = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
plt.figure(figsize=(20, 15))
num_row = 4
num_fig_per_row = len(demand_perct) / num_row
subplot_idx = 1
plt.rcParams.update({'font.size': 20})

for demand in demand_perct:
    energy_saving_sig_opt = energy_saving.loc[energy_saving['demand'] == demand, 'signal_optimization']
    energy_saving_eco_drive = energy_saving.loc[energy_saving['demand'] == demand, 'eco-drive']
    energy_saving_cooperative_control = energy_saving.loc[energy_saving['demand'] == demand, 'cooperative_control']
    
    plt.subplot(num_row, num_fig_per_row, subplot_idx)
    plt.plot(cav_perct, energy_saving_sig_opt, 'r')
    plt.plot(cav_perct, energy_saving_eco_drive, 'b')
    plt.plot(cav_perct,energy_cooperative_control, 'g')
    plt.xlabel('CAV market penetration')
    plt.title('Demand {}'.format(demand))

    if subplot_idx % 4 == 1:
        plt.ylabel('Energy saving relative to base case (%)')
    
    if subplot_idx == len(demand_perct):
        plt.legend(['Signal optimization only', 'Eco-driving only', 'Cooperative control'])
    
    subplot_idx += 1

# energy saving vs. demand
cav_perct = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
plt.figure(figsize=(20, 15))
num_row = 5
num_fig_per_row = len(cav_perct) / num_row
subplot_idx = 1
plt.rcParams.update({'font.size': 20})

for cav in cav_perct:
    energy_saving_sig_opt = energy_saving.loc[energy_saving['cav'] == cav, 'signal_optimization']
    energy_saving_eco_drive = energy_saving.loc[energy_saving['cav'] == cav, 'eco-drive']
    energy_saving_cooperative_control = energy_saving.loc[energy_saving['cav'] == cav, 'cooperative_control']
    
    plt.subplot(num_row, num_fig_per_row, subplot_idx)
    plt.plot(demand_perct, energy_saving_sig_opt, 'r')
    plt.plot(demand_perct, energy_saving_eco_drive, 'b')
    plt.plot(demand_perct,energy_cooperative_control, 'g')
    plt.xlabel('Demand levels (%)')
    plt.title('CAV market penetration {}'.format(cav))

    if subplot_idx % 2 == 1:
        plt.ylabel('Energy saving relative to base case (%)')
    
    if subplot_idx == len(cav_perct):
        plt.legend(['Signal optimization only', 'Eco-driving only', 'Cooperative control'])
    
    subplot_idx += 1